In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from numpy import unique
from numpy import reshape
from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, Dense, BatchNormalization, Flatten, MaxPooling1D
from keras.layers.core import Dense, Dropout
from keras.utils import np_utils

In [2]:
path = r'/Users/yourname/Datasets'

In [4]:
tf.random.set_seed(42)

In [5]:
signals = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [6]:
activities = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

In [7]:
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

def load_signals(subset):
    signals_data = []

    for signal in signals:
        filename = f'{path}/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).to_numpy()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'{path}/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).to_numpy()

def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [8]:
X_train, X_test, Y_train, Y_test = load_data()

In [9]:
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(7352, 128, 9) (7352, 6)
(2947, 128, 9) (2947, 6)


In [10]:
X_train

array([[[ 1.808515e-04,  1.076681e-02,  5.556068e-02, ...,
          1.012817e+00, -1.232167e-01,  1.029341e-01],
        [ 1.013856e-02,  6.579480e-03,  5.512483e-02, ...,
          1.022833e+00, -1.268756e-01,  1.056872e-01],
        [ 9.275574e-03,  8.928878e-03,  4.840473e-02, ...,
          1.022028e+00, -1.240037e-01,  1.021025e-01],
        ...,
        [-1.147484e-03,  1.714439e-04,  2.647864e-03, ...,
          1.018445e+00, -1.240696e-01,  1.003852e-01],
        [-2.222655e-04,  1.574181e-03,  2.381057e-03, ...,
          1.019372e+00, -1.227451e-01,  9.987355e-02],
        [ 1.575500e-03,  3.070189e-03, -2.269757e-03, ...,
          1.021171e+00, -1.213260e-01,  9.498741e-02]],

       [[ 1.093752e-03, -4.687588e-03, -2.685954e-02, ...,
          1.018851e+00, -1.239760e-01,  9.792958e-02],
        [ 4.550077e-03, -7.487894e-03, -2.509841e-02, ...,
          1.022380e+00, -1.268078e-01,  9.935086e-02],
        [ 2.879173e-03, -8.429991e-03, -2.597534e-02, ...,
          1.02

In [11]:
len(X_train[0])

128

In [12]:
len(X_train[0][0])

9

In [13]:
epochs = 30
batch_size = 16
n_hidden = 32

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) #sigmoid

In [14]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
460/460 - 1s - loss: 0.5563 - accuracy: 0.7813 - 847ms/epoch - 2ms/step
Epoch 2/30
460/460 - 0s - loss: 0.2325 - accuracy: 0.9203 - 454ms/epoch - 987us/step
Epoch 3/30
460/460 - 1s - loss: 0.1601 - accuracy: 0.9410 - 505ms/epoch - 1ms/step
Epoch 4/30
460/460 - 0s - loss: 0.1337 - accuracy: 0.9480 - 491ms/epoch - 1ms/step
Epoch 5/30
460/460 - 0s - loss: 0.1202 - accuracy: 0.9495 - 460ms/epoch - 1ms/step
Epoch 6/30
460/460 - 0s - loss: 0.1135 - accuracy: 0.9531 - 460ms/epoch - 1000us/step
Epoch 7/30
460/460 - 0s - loss: 0.1070 - accuracy: 0.9535 - 490ms/epoch - 1ms/step
Epoch 8/30
460/460 - 0s - loss: 0.1049 - accuracy: 0.9554 - 498ms/epoch - 1ms/step
Epoch 9/30
460/460 - 0s - loss: 0.0979 - accuracy: 0.9572 - 441ms/epoch - 959us/step
Epoch 10/30
460/460 - 0s - loss: 0.0933 - accuracy: 0.9585 - 457ms/epoch - 993us/step
Epoch 11/30
460/460 - 0s - loss: 0.0911 - accuracy: 0.9600 - 451ms/epoch - 980us/step
Epoch 12/30
460/460 - 0s - loss: 0.0872 - accuracy: 0.9607 - 457ms/epoch -

In [16]:
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([activities[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([activities[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

In [17]:
# Evaluate
print(confusion_matrix(Y_test, model.predict(X_test)))

93/93 [==============================] - 0s 630us/step
Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 494        0         0        0                   0   
SITTING                  5      413        53        0                   0   
STANDING                 0      113       415        1                   0   
WALKING                  0        0         0      450                  28   
WALKING_DOWNSTAIRS       0        0         1        8                 406   
WALKING_UPSTAIRS         0        0         0       19                  34   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            43  
SITTING                           20  
STANDING                           3  
WALKING                           18  
WALKING_DOWNSTAIRS                 5  
WALKING_UPSTAIRS                 418  
